# Topic modeling

We are going to look at data from the [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/) dataset.  These are postings to newsgroups in 20 different categories.

Scikit-learn has a function for downloading the data.  See: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

## Supervised Learning with Naive Bayes

We are going to start out using Naive Bayes for supervised ML.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
ntrain = fetch_20newsgroups(subset='train')

Here is the description of the dataset, along with an interesting example and tidbits of advice: 

In [ ]:
print(ntrain.DESCR)

An example of the first five newsgroup posts:

In [ ]:
ntrain.data[:5]

We have 11314 newsgroup postings, with 20 classes.

In [ ]:
ntrain.target.shape

In [ ]:
ntrain.target[:10]

In [ ]:
[ntrain.target_names[i] for i in ntrain.target[:10]]

We'll keep it a bit simpler for the moment and only consider two of the categories:

In [ ]:
cats = ['sci.space', 'comp.graphics']

Re-import the training data for just these two categories, along with test data too.

In [ ]:
ntrain = fetch_20newsgroups(subset='train', categories=cats)
ntest = fetch_20newsgroups(subset='test',categories=cats)

In [ ]:
print(ntrain.data[0])

How does this compare with what I showed on the slide?  And does it matter that it's different?

In [ ]:
ntrain.target_names

In [ ]:
ntrain.target.shape

In [ ]:
ntrain.target[:10]

In [ ]:
ntest.target.shape

We can choose how to convert our posts and words into a document-term matrix.  Let's use TF-IDF:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(ntrain.data)
vectors_test = vectorizer.transform(ntest.data)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(vectors_train.toarray(),
                  columns=vectorizer.get_feature_names_out())

In [ ]:
df

What do the number of rows and columns in the dataframe represent?

In [ ]:
df[['earth','graphics','image','nasa','algorithms','astronomy']]

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
clf = MultinomialNB()
clf.fit(vectors_train, ntrain.target)

In [ ]:
pred = clf.predict(vectors_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
metrics.accuracy_score(ntest.target, pred)

In [ ]:
print(classification_report(ntest.target, pred))

So good!

Too good??

Let's look at the words that have the highest probability for each of these two classes.

We can get the feature names with:

In [ ]:
vectorizer.get_feature_names_out()

And we can get the probabilities (actually log of probabilities because they're so small) with:

In [ ]:
clf.feature_log_prob_

In [ ]:
clf.feature_log_prob_.shape

There are two rows and 23882 columns because this shows *P(class | word)* for each class and each word.

In [ ]:
import numpy as np

In [ ]:
def show_top10(classifier, vectorizer, categories):
    feature_names = vectorizer.get_feature_names_out()
    for i,category in enumerate(categories):
        top10 = np.argsort(-classifier.feature_log_prob_[i])[:10]
        print('%s:' % (category))
        for j in top10:
            print("%s: %.2f" % (feature_names[j], classifier.feature_log_prob_[i][j]))
        print('\n')

In [ ]:
show_top10(clf, vectorizer, ntrain.target_names)

We could stand to do more data processing.

First, remove all the meta-data in the headers and footers, along with quotes.

In [ ]:
ntrain = fetch_20newsgroups(subset='train',categories=cats,remove=('headers','footers','quotes'))
ntest = fetch_20newsgroups(subset='test',categories=cats,remove=('headers','footers','quotes'))

In [ ]:
vectors = vectorizer.fit_transform(ntrain.data)
vectors_test = vectorizer.transform(ntest.data)

In [ ]:
clf = MultinomialNB()
clf.fit(vectors, ntrain.target)
pred = clf.predict(vectors_test)

In [ ]:
print(classification_report(ntest.target, pred))

The performance dropped a little, but likely because we aren't overfitting to words in the parts of the post that aren't as relevant to our meaningful newsgroup content.

How do the word probabilities change?

In [ ]:
show_top10(clf, vectorizer, ntrain.target_names)

Still have a lot of fluff.

We can remove stopwords.  The TfidfVectorizer will also allow us to remove words that are really really common, as well as requiring words to occur at least a certain minimum number of times:

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=2)

In [ ]:
vectors = vectorizer.fit_transform(ntrain.data)
vectors_test = vectorizer.transform(ntest.data)

In [ ]:
clf = MultinomialNB()
clf.fit(vectors, ntrain.target)
pred = clf.predict(vectors_test)

In [ ]:
print(classification_report(ntest.target, pred))

The performance did not drop; it may even have slightly improved.

In [ ]:
show_top10(clf, vectorizer, ntrain.target_names)

That is looking like a much more meaningful set of words.

Let me try to make a prediction on a new (hypothetical) newsgroup post.

In [ ]:
mypost = '''
I was sitting outside with my cat, looking up at the sky through our telescope, 
when lo and behold, to my little set of eyes I spied an anomalous signal emanating 
from a distant galaxy.  I've scoured my astronomy books and can't find any 
description of what I saw.  Is this a new type of celestial phenomenon? or, 
dare I say it, something extraterrestrial in origin?

Please let me know if you're up for sharing insights on my dataset.
'''

Do remember that if you want to make a prediction on a new data point, you'll need to transform it in the same way that you transformed your training data.

In [ ]:
vectorizer.transform([mypost])

In [ ]:
vectorizer.transform([mypost]).toarray()

So many zeroes, we can't really see much in looking at our vector representation.

In [ ]:
clf.predict(vectorizer.transform([mypost]))

In [ ]:
ntrain.target_names[1]

After all the transforming, predictions, and decoding, indeed we find that the approach correctly classifies mypost as belonging to sci.space.

.... though with a certain probability:

In [ ]:
clf.predict_proba(vectorizer.transform([mypost]))